In [1]:
import  torch

In [7]:
import  pandas as pd
import  numpy as np
from    tqdm import tqdm
import  time

import  rdflib
from    sklearn.metrics import precision_score, recall_score, f1_score



In [4]:
import sys
sys.path.append("..") 

In [9]:
from    src.utils import *
from    src.gnn import *

In [10]:
# Load Data

g = rdflib.Graph()
g.parse('../data/oaei/h2m_map_typed.ttl')

print(f'Triplets found: %d' % len(g))

Triplets found: 7101


In [11]:
relations = list(set(g.predicates()))
nodes = list(set(g.subjects()).union(set(g.objects())))

relations_dict = {rel: i for i, rel in enumerate(relations)}
nodes_dict = {node: i for i, node in enumerate(nodes)}

nodes_dict_rev = {value: key for key, value in nodes_dict.items()}
relations_dict_rev = {value: key for key, value in relations_dict.items()}

In [12]:
data = get_data(g, nodes_dict, relations_dict)
data = split_edges(data)

In [13]:
data

HeteroData(
  edge_index=[2, 7101],
  edge_type=[7101],
  val_pos_edge_index=[2, 0],
  val_edge_type=[0],
  test_pos_edge_index=[2, 1420],
  test_edge_type=[1420],
  train_pos_edge_index=[2, 5681],
  train_edge_type=[5681]
)

In [15]:
st = time.time()
model = GNN()

for epoch in range(100+1):
    loss = model._train(data, len(nodes), len(relations))
    if (epoch % 100) == 0:
        hits1, hits10 = model._eval(data)
        print(f'Epoch: {epoch}, Loss: {loss:.4f},'+ 
              f'Hits@1: {hits1:.3f}, Hits@10: {hits10:.3f}')

torch.save(model, f'../models/RGCN_h2m_maps')
et = time.time()
elapsed_time = et - st
print(f'Run time: {elapsed_time:.0f} seconds, {elapsed_time/60:.0f} minutes')

Epoch: 0, Loss: 0.6932,Hits@1: 0.817, Hits@10: 0.896
Epoch: 100, Loss: 0.6932,Hits@1: 0.832, Hits@10: 0.904
Run time: 142 seconds, 2 minutes
